# Design Patterns
(**Caution:** May contain Java)

## Let's start with an example

**Problem:** We have multiple components that want to get updated information from a single source

### Approach 1:
The various components periodically contact source for updates

![alt](polling.svg)

# Approach 2:
Publisher pushes updates to subscribers whenever it has updates.

![alt](push.svg)

In [2]:
class Publisher:
    def __init__(self):
        self.subscribers = []
    def add_subscriber(self, subscriber):
        self.subscribers.append(subscriber)
    def publish(self, data):
        for subscriber in self.subscribers:
            subscriber.notify(data)
        
class Subscriber:
    def notify(self, data):
        pass

### Observer Pattern 
*(a.k.a Event-Subscriber, Publisher-Subscriber Listener)*

Publisher pushed updates to subscriber avoiding overhead of unecessary polling.

## So what are patterns anyway?

1. Provide receipes for solving common problems
1. Define a vocabulary for discussing software structure

## Resources

* https://pyvideo.org/pyohio-2012/python-design-patterns-1.html (first 17 minutes is repeated so start at the 17 minute mark)
* https://python-patterns.guide/
* https://refactoring.guru
* https://diagrams.mingrammer.com/

## Notes from Brandon's presentation:

https://www.youtube.com/watch?v=Er5K_nR5lDQ
(start of video is repeated if you start 17 minutes in you can skip the repeated section)

3 reasons that we don't talk about patterns much in Python: 
 1. patterns are baked into the Language and the standard library
 1. provided by frameworks (i.e. Django & MVC)
 1. we avoid large complex applications
 
Book "A Pattern Language" Christopher Alexander (1977) on Architecture (buildings)
 
Book: "Design Patterns - Elements of Reusable Object-oriented Software" Gamma, Helm, Johnson & Vlissides (The Gang of Four)

not all patterns are Object-oriented patterns.  Using a_dict.get() instead of checking if key exists first.  Avoiding elses in code.

OO Design patterns maxim: "Program to an interface not an implementation".  With duck-typing in Python, this is largely automatic.

Pattern Types:
1. Creational
1. Structural
1. Behavioral

### Creational Patterns

No separate syntax for creating instances of a class (no new keyword)

having a new keywords means you can't do things like:
1. return another class
1. Always return the same instance.

"So most 'creational patterns' are contortions to avoid making brittle new calls in your C++/Java code"

Prototype pattern - Creating new objects based on existing objects.  The standard library's copy module takes care of this for us.

Builder pattern - you feed the builder info on what you want built and it takes care of the gory details of linking the various instances together.  Brandon has an example of building an XML document. At about 40 minutes in.

Left off at 40 min in

Designed away by language: Abstract Factory, factory method
Trivial: Singleton, Prototype
Useful: Builder

### Structural Patterns

**Adapter:** Wrap a class so it behaves like another.  i.e. wrap a socket that it has a `read` and `write` method like a file instead of the socket's normal `send` and `recv` methods.  (43 min)

**Bridge:** Use subclass for 1 purpose only.  You don't want a bunch of classes like:
* WindowsButton
* WindowsBorderedButton
* LinuxButton
* LinuxBorderedButton

Mixins may allow your better way to achieve this

**Composite:** A class whose instances are designed to lock together in a tree.  Like `Node` elements in a Document Object Model.  Very common in the Python World

**Fascade:** An object that hides a complex tree or network of objects.  i.e. etrees root.find() and root.iter() that handle the details of searching and iterating through the tree structure.

**Flyweight:** small immutable object that may be used in a variety of contexts.
(I think named tuples would be a good way to accomplish this)

left off at 55 minutes right as the proxy pattern was introduced




In [7]:
# Brandon's take on Python singleton
# Don't use a module-level variable (i.e. my_module.foo) that doesn't leave 
# you any room if you need to do something in more than just a variable.
class MyClass: pass

_singleton = MyClass()
def get_singleton():
    return _singleton

# "Real" singletons allow caller to specify a subclass the first time the 
# singleton is retrieved.  A gang of four singleton looks like this:

_other_singleton = None
def get_other_singleton(cls=MyClass):
    global _other_singleton
    if _other_singleton is None:
        _other_singleton = cls()
    return _other_singleton